In [1]:
import dlib
import cv2
import numpy as np

#shape_predictor_68_face_landmarks.dat

In [2]:
cap = cv2.VideoCapture(0)
cap.set(3,720)
cap.set(4,1020)

True

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
def empty(a):
    pass

In [5]:
cv2.namedWindow('BGR')
cv2.resizeWindow('BGR',640,240)
cv2.createTrackbar('Blue','BGR',0,255,empty)
cv2.createTrackbar('Green','BGR',0,255,empty)
cv2.createTrackbar('Red','BGR',0,255,empty)

In [6]:
def createBox(img,points,scale=5,masked=False,cropped = True):
    if masked:
        mask = np.zeros_like(img)
        mask = cv2.fillPoly(mask,[points],(255,255,255))
        img = cv2.bitwise_and(img,mask)

    if cropped:
        bbox = cv2.boundingRect(points)
        x,y,w,h = bbox
        imgCrop = img[y:y+h,x:x+w]
        imgCrop = cv2.resize(imgCrop,(0,0),None,scale,scale)
        return imgCrop
    else:
        return mask

In [7]:
while True:
    success, img = cap.read()
    img = cv2.resize(img,(0,0),None,1,1)
    imgOriginal = img.copy()
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(imgGray)
    
    for face in faces:
        x1,y1 = face.left(),face.top()
        x2,y2 = face.right(),face.bottom()
        landmarks = predictor(imgGray,face)
        myPoints =[]
        for n in range(68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            myPoints.append([x,y])
        myPoints = np.array(myPoints)
        
        imgLips = createBox(img,myPoints[48:61],8,masked=True,cropped=False)

        imgColorLips = np.zeros_like(imgLips)
        b = cv2.getTrackbarPos('Blue','BGR')
        g = cv2.getTrackbarPos('Green','BGR')
        r = cv2.getTrackbarPos('Red','BGR')
        imgColorLips[:] = b,g,r
        imgColorLips = cv2.bitwise_and(imgLips,imgColorLips)
        imgColorLips = cv2.GaussianBlur(imgColorLips,(7,7),10)
        
        imgColorLips = cv2.addWeighted(imgOriginal,1,imgColorLips,0.4,0)
        
        cv2.imshow('BGR',imgColorLips)

    if cv2.waitKey(1) == ord('q'):
        break